# QianfanChatEndpoint | QianfanChatEndpointBaidu AI Cloud Qianfan Platformは、企業開発者向けのワンストップ大規模モデル開発およびサービス運用プラットフォームです。Qianfanは、Wenxin Yiyan（ERNIE-Bot）のモデルやサードパーティのオープンソースモデルを含むだけでなく、様々なAI開発ツールや完全な開発環境のセットも提供し、顧客が大規模モデルアプリケーションを容易に使用し、開発できるよう支援します。> Baidu AI Cloud Qianfan Platform is a one-stop large model development and service operation platform for enterprise developers. Qianfan not only provides including the model of Wenxin Yiyan (ERNIE-Bot) and the third-party open-source models, but also provides various AI development tools and the whole set of development environment, which facilitates customers to use and develop large model applications easily.基本的に、それらのモデルは以下のタイプに分けられます：> Basically, those model are split into the following type:* 埋め込み  > Embedding* チャット  > Chat* 完了  > Completionこのノートブックでは、`langchain/chat_models`パッケージに相当する`Chat`を主に使用して、[Qianfan](https://cloud.baidu.com/doc/WENXINWORKSHOP/index.html)と共にlangchainの使用方法を紹介します。> In this notebook, we will introduce how to use langchain with [Qianfan](https://cloud.baidu.com/doc/WENXINWORKSHOP/index.html) mainly in `Chat` corresponding> to the package `langchain/chat_models` in langchain:## API Initialization | APIの初期化Baidu Qianfanに基づくLLMサービスを使用するには、これらのパラメータを初期化する必要があります：> To use the LLM services based on Baidu Qianfan, you have to initialize these parameters:AKとSKを環境変数で初期化するか、初期化パラメータで初期化するか選択できます：> You could either choose to init the AK,SK in environment variables or init params:```baseexport QIANFAN_AK=XXXexport QIANFAN_SK=XXX```## Current supported models: | 現在サポートされているモデル：* ERNIE-Bot-turbo（デフォルトモデル）  > ERNIE-Bot-turbo (default models)* ERNIE-Bot  > ERNIE-Bot* BLOOMZ-7B  > BLOOMZ-7B* Llama-2-7b-chat  > Llama-2-7b-chat* Llama-2-13b-chat  > Llama-2-13b-chat* Llama-2-70b-chat  > Llama-2-70b-chat* Qianfan-BLOOMZ-7B-compressed  > Qianfan-BLOOMZ-7B-compressed* Qianfan-Chinese-Llama-2-7B  > Qianfan-Chinese-Llama-2-7B* ChatGLM2-6B-32K  > ChatGLM2-6B-32K* AquilaChat-7B  > AquilaChat-7B

In [5]:
"""For basic init and call"""
import os

from langchain.chat_models import QianfanChatEndpoint
from langchain_core.language_models.chat_models import HumanMessage

os.environ["QIANFAN_AK"] = "your_ak"
os.environ["QIANFAN_SK"] = "your_sk"

chat = QianfanChatEndpoint(
    streaming=True,
)
res = chat([HumanMessage(content="write a funny joke")])

[INFO] [09-15 20:00:29] logging.py:55 [t:139698882193216]: requesting llm api endpoint: /chat/eb-instant


In [6]:
from langchain.chat_models import QianfanChatEndpoint
from langchain.schema import HumanMessage

chatLLM = QianfanChatEndpoint(
    streaming=True,
)
res = chatLLM.stream([HumanMessage(content="hi")], streaming=True)
for r in res:
    print("chat resp:", r)


async def run_aio_generate():
    resp = await chatLLM.agenerate(
        messages=[[HumanMessage(content="write a 20 words sentence about sea.")]]
    )
    print(resp)


await run_aio_generate()


async def run_aio_stream():
    async for res in chatLLM.astream(
        [HumanMessage(content="write a 20 words sentence about sea.")]
    ):
        print("astream", res)


await run_aio_stream()

[INFO] [09-15 20:00:36] logging.py:55 [t:139698882193216]: requesting llm api endpoint: /chat/eb-instant
[INFO] [09-15 20:00:37] logging.py:55 [t:139698882193216]: async requesting llm api endpoint: /chat/eb-instant


chat resp: content='您好，您似乎输入' additional_kwargs={} example=False
chat resp: content='了一个话题标签，请问需要我帮您找到什么资料或者帮助您解答什么问题吗？' additional_kwargs={} example=False
chat resp: content='' additional_kwargs={} example=False


[INFO] [09-15 20:00:39] logging.py:55 [t:139698882193216]: async requesting llm api endpoint: /chat/eb-instant


generations=[[ChatGeneration(text="The sea is a vast expanse of water that covers much of the Earth's surface. It is a source of travel, trade, and entertainment, and is also a place of scientific exploration and marine conservation. The sea is an important part of our world, and we should cherish and protect it.", generation_info={'finish_reason': 'finished'}, message=AIMessage(content="The sea is a vast expanse of water that covers much of the Earth's surface. It is a source of travel, trade, and entertainment, and is also a place of scientific exploration and marine conservation. The sea is an important part of our world, and we should cherish and protect it.", additional_kwargs={}, example=False))]] llm_output={} run=[RunInfo(run_id=UUID('d48160a6-5960-4c1d-8a0e-90e6b51a209b'))]
astream content='The sea is a vast' additional_kwargs={} example=False
astream content=' expanse of water, a place of mystery and adventure. It is the source of many cultures and civilizations, and a center

## Use different models in Qianfan | Qianfanで異なるモデルを使用するErnie Botやサードパーティのオープンソースモデルに基づいて自分のモデルをデプロイしたい場合は、以下の手順に従ってください：> In the case you want to deploy your own model based on Ernie Bot or third-party open-source model, you could follow these steps:* 1. （オプショナル、もしモデルがデフォルトモデルに含まれている場合はこれをスキップしてください）Qianfanコンソールであなたのモデルをデプロイし、あなた専用のカスタマイズされたデプロイエンドポイントを取得してください。     > （Optional, if the model are included in the default models, skip it）Deploy your model in Qianfan Console, get your own customized deploy endpoint.* 2. 初期化の際に`endpoint`というフィールドを設定してください。     > Set up the field called `endpoint` in the initialization:

In [7]:
chatBloom = QianfanChatEndpoint(
    streaming=True,
    model="BLOOMZ-7B",
)
res = chatBloom([HumanMessage(content="hi")])
print(res)

[INFO] [09-15 20:00:50] logging.py:55 [t:139698882193216]: requesting llm api endpoint: /chat/bloomz_7b1


content='你好！很高兴见到你。' additional_kwargs={} example=False


## Model Params: | モデルパラメータ:現時点では、`ERNIE-Bot` と `ERNIE-Bot-turbo` のみが以下に示すモデルパラメータをサポートしていますが、将来的にはさらに多くのモデルをサポートする可能性があります。> For now, only `ERNIE-Bot` and `ERNIE-Bot-turbo` support model params below, we might support more models in the future.* 温度  > temperature* top\_p  > top\_p* ペナルティスコア  > penalty\_score

In [8]:
res = chat.stream(
    [HumanMessage(content="hi")],
    **{"top_p": 0.4, "temperature": 0.1, "penalty_score": 1},
)

for r in res:
    print(r)

[INFO] [09-15 20:00:57] logging.py:55 [t:139698882193216]: requesting llm api endpoint: /chat/eb-instant


content='您好，您似乎输入' additional_kwargs={} example=False
content='了一个文本字符串，但并没有给出具体的问题或场景。' additional_kwargs={} example=False
content='如果您能提供更多信息，我可以更好地回答您的问题。' additional_kwargs={} example=False
content='' additional_kwargs={} example=False
